In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error

## Cross-validation

Previously, we've seen why it's important to split our dataset into a train and a test split, in order to more fairly assess the usefullness of our model. We saw that the performance metrics (loss) on the train data is, in itself, a bad assessement of this and that good performance on the train set does not imply good performance on the test set.

In any case, we've also seen now that even this data splitting strategy has its limits, since we sometime can get lucky (or unlucky, depending on how you look at it) with how this split occurs. If we get a particularly bad split, we might obtain really good performance on the test set on models that otherwise are severly overfit or underfit to our data.

To bypass this, we'll be using **k-fold cross-validation**. 

This is a very simple generalisation of the train and test split strategy, where we just construct *k* seperate splits of our full dataset into *k* different train and test splits.

The idea is then to fit our model (with the chosen features) on each of these *k* different train/test splits and average the performance over each. This newly obtained perfromance average is then usually a much better assessement of the model's (with the given features) true performance.

Simply put, here are the steps for *k*-fold cross validation:

    1. 
Construct *k* different train and test splits from our full data. 
    
    2. 
Train our model, with the given features, on each of the *k* different train and test splits.
    
    3. 
Calculate the performance metric after training (could e.g., be MAE).
    
    4. 
Average the MAE over the *k* different training runs.

Usual values for *k* is somwhere around 5 or 10. Chosing larger values for *k* provides better estimates, but brings about larger compute costs.

---

**Implementation**

Let's first import our dataset. We'll use the bike demand dataset from the previous lab.

We'll skip all the data analysis, data cleaning and feature engineering here so that we can
showcase cross-validation instead.

In [ ]:
df = pd.read_csv('../data/bike_rental/day.csv')

df

In [ ]:
# drop unwanted and forbidden columns (why are casual and registered forbidden here?)

df = df.drop(columns=['instant', 'dteday', 'casual', 'registered'])

df

In [ ]:
# seperate features and target

X, y = df.drop(columns=['cnt']), df['cnt']

**Excplicit implementation of k-fold cross-validation**

Here's an explicit implementation of k-fold cross validation

In [ ]:
# Initialize list to store MAE values
mae_values = []

# Define the number of folds
num_folds = 10

# Loop through each fold

for fold in range(num_folds):
    
    # for each fold, create a random train/test split

    # NOTE, there are different philosophies for how to chose the test size here. 
    
    # 1. We can keep it constant throughout each fold or

    # 2. As some like to do it, choose the test size to be 1/num_folds
    # chosing num_folds = 5 then means a test size of 0.2 = 20%. 
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/num_folds)
    
    # Initialize linear regression model
    model = LinearRegression()
    
    # Train the model using X_train and y_train
    model.fit(X_train, y_train)
    
    # Make predictions
    y_test_hats = model.predict(X_test)
    
    # Calculate Root Mean Absolute Error
    mae = mean_absolute_error(y_test, y_test_hats)
    
    # Append MSE to list
    mae_values.append(mae)

# Print MAE values for each fold
for i, mae in enumerate(mae_values):
    print(f"Fold {i+1} MAE: {mae}")

print()
print('Average MAE:', np.mean(mae_values))

**Make it a function**

In [ ]:
from sklearn.metrics import mean_absolute_error


def my_cross_validator(model, X, y, evaluation_function, num_folds, test_size):

    loss_test_values = []
    loss_train_values = []

    for fold in range(num_folds):

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)   # notera att detta skapar en helt random uppdelning av train/test split
                                                                                         # för varje loop - vilket är precis vad vi vill.
        model.fit(X_train, y_train)

        y_test_hats = model.predict(X_test)
        y_train_hats = model.predict(X_train)

        test_loss = evaluation_function(y_test, y_test_hats)
        train_loss = evaluation_function(y_train, y_train_hats)

        loss_test_values.append(test_loss)
        loss_train_values.append(train_loss)

    return loss_test_values, loss_train_values

In [ ]:
model = LinearRegression()
num_folds = 5
test_size = 0.2

test_losses, train_losses = my_cross_validator(model, X, y, mean_absolute_error, num_folds, test_size)


print('test:')
print(test_losses, end='\n\n')
print('test average:')
print(np.mean(test_losses), end='\n\n')

print('train:')
print(train_losses, end='\n\n' )
print('train average:')
print(np.mean(train_losses))

In [ ]:
import matplotlib.pyplot as plt

plt.hist(y, bins=30)
plt.show()

**Using Sklearn**

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error


# Initialize linear regression model
model = LinearRegression()

# Define the scoring function
scoring = {'mae': make_scorer(mean_absolute_error),
           'mse': make_scorer(mean_squared_error)}

# Perform 5-fold cross-validation
num_folds = 5

cv_results = cross_validate(model, X, y, cv=num_folds, scoring=scoring, return_train_score=True)

In [ ]:
# Extract MAE values for each fold
mae_values = cv_results['test_mae']

# Print MAE values for each fold
for i, mae in enumerate(mae_values):
    print(f"Fold {i+1} MAE: {mae}")

print()
print('Average MAE:', np.mean(cv_results['test_mae']))

**To learn more about sklearns own cross-validators, read the documentation**

You can either use:

1. The function **cross_validation**, which we used above
Documentation [here.](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)

2. The function **cross_val_score**
Documentation [here.](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)

---

**How to use this**

We use cross validation to assess the perfromance of *every* model and feature(s) pair!

For example, assume we have two models, each called model_1 and model_2, respectively.

Assume also that we now have features $x_1, x_2, x_3, x_4$.

We now want to find out which model and feature combination that produces the pest result on our data.

For starters, say that we want to assess model_1 combined with features $x_1, x_2$ and $x_3$.

We then do a full 5-fold cross-validation on this model with this features. In other words:

Let X consist of $x_1, x_2, x_3$.

Then, we simply

cv_results = cross_validate(model_1, X, y, cv=5, scoring=scoring).

and therafter calculate

combination_average_performance = np.mean(cv_results['test_mae'])

This resulting average is what we use to compare performance with any other model and feature combination!


---

## Challanges

**Task**

Try using cross-validation to assess performance on the bike demand dataset using a linear model combined with different sets of features.

You can use whichever method to cross-valdiate we've shown above, as you like.

In [ ]:
df = pd.read_csv('../data/bike_rental/day.csv')